In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point

### Loading data

#### Airports

In [3]:
# load data
airports_raw = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', header = None) #load data


# Assign column names
airports_raw.columns = ["Airport_id","Name","City","Country","3DigitId","4DigitId","Lat","Lon","Altitude_ft","Timezone","DST","TZ","Type","Source"] 

# Pick relevant columns for analysis
airports_df = airports_raw[["Airport_id","Name","City","Country","3DigitId","4DigitId","Lat","Lon","Altitude_ft","Type"]]

In [6]:
# create geopandas dataframe
airports_df["Coordinates"] = list(zip(airports_df.Lon, airports_df.Lat))
airports_df.Coordinates = airports_df.Coordinates.apply(Point)
airports_gdf = gpd.GeoDataFrame(airports_df, geometry = "Coordinates")
airports_gdf.crs = {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True}

C:\Users\mra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\mra\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


#### Connections ("routes")

In [7]:
connections_raw = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat', header = None)

connections_raw.columns = ["Airline_IATA/ICAO",
                           "Airline_ID",
                           "Source_airport","Source_airport_ID",
                           "Destination_airport","Destination_airport_ID",
                          "Codeshare",
                          "Stops",
                          "Equipment"]

#### Indicator for which airports are in the connections data

In [8]:
# Make indicator in the airports dataset, for whether the airport is included in the connections dataset
airports_gdf["Airport_id"] = airports_gdf["Airport_id"].astype(str)
connections_raw['Source_airport_ID'] = connections_raw['Source_airport_ID'].astype(str)
airports_gdf["Connected"] = airports_gdf.Airport_id.isin(list(connections_raw.Destination_airport_ID))
airports_gdf.Connected.sum()

3184

#### Pickle the two datasets

In [11]:
airports_gdf.to_pickle("airports.pkl")
connections_raw.to_pickle("connections.pkl")